In [1]:
from pydbgen import pydbgen
import os
import pandas as pd
from faker import Faker
fake = Faker()
import random
import numpy as np
import string
import os
from tqdm import tqdm
import getpass

In [ ]:
uname = 'postgres'
passwd = getpass.getpass()

# Postgres Test

In [40]:
import psycopg2

In [41]:
def table_exists(con, table_str):

    exists = False
    try:
        cur = con.cursor()
        cur.execute("select exists(select relname from pg_class where relname='" + table_str + "')")
        exists = cur.fetchone()[0]
        print (exists)
        cur.close()
    except psycopg2.Error as e:
        print (e)
    return exists

def get_table_col_names(con, table_str):

    col_names = []
    try:
        cur = con.cursor()
        cur.execute("select * from " + table_str + " LIMIT 0")
        for desc in cur.description:
            col_names.append(desc[0])        
        cur.close()
    except psycopg2.Error as e:
        print (e)

    return col_names

In [42]:
conn = psycopg2.connect(f"host='127.0.0.1' dbname='synthetic_email' user='{uname}' password='{passwd}'")

In [43]:
cur = conn.cursor()

In [44]:
cur.execute('SELECT count(*) FROM employees')

In [45]:
size = cur.fetchone()

In [191]:
size[0]

48801

In [ ]:
!head /encrypted_vol/jupyter_data/airflow/data/employees.csv

In [192]:
cur.execute('DROP TABLE IF EXISTS employees')

In [74]:
#if size[0] < 48000: # only execute if we are missing data
cur.execute('DROP TABLE IF EXISTS employees')
cur.execute('''CREATE TABLE employees (
            name varchar,
            email varchar, 
            phone_number varchar,
            address varchar,
            PRIMARY KEY (name, email));''')



cur.execute("""COPY employees(name,email,phone_number,address) 
                FROM '/encrypted_vol/jupyter_data/airflow/data/employees.csv'
                DELIMITER ',' CSV HEADER;""")

In [75]:
cur.execute('SELECT * FROM employees')

In [76]:
cur.fetchone()

('Chad Spears',
 'zacharyerickson@yahoo.com',
 '557-303-4631',
 '742 Jessica Cliffs, Port Stephenstad, TN 06314')

### Insert into email log table

In [117]:
!head -n 2 /encrypted_vol/jupyter_data/airflow/data/2019-08-23.csv

guid,datetime,sender,recipients,subject,attachments,rule,action,attach_cnt,rcpt_cnt,email_size,log_file,log_key_id
aRFWSXGwyJiTxPGMwzZSJcdRjWtpjCqB,2019-08-23 00:00:01+00:00,nsanchez@gilbert-wade.com,"['rodriguezsabrina@yahoo.com', 'ymcbride@hotmail.com', 'gberry@hotmail.com', 'karen09@hotmail.com']",Position or expect someone radio have.,['H:/him/technology/memory/many/sport/claim.inf'],AND,WHOLE_MAINTAIN_STOP,1,4,26100063,201901.gz,


In [46]:
cur.execute('DROP TABLE IF EXISTS email_logs')

In [47]:
cur.execute('''CREATE TABLE IF NOT EXISTS email_logs 
            (guid varchar, 
            "datetime" TIMESTAMP, 
            sender varchar, 
            recipients varchar, 
            subject varchar, 
            attachments varchar, 
            rule varchar, 
            action varchar, 
            attach_cnt int, 
            rcpt_cnt int, 
            email_size int, 
            log_file varchar, 
            log_key_id varchar,
            PRIMARY KEY (guid, sender));''')

In [94]:
cur.execute('''COPY email_logs(guid,datetime,sender,recipients,subject,attachments,rule,action,attach_cnt,rcpt_cnt,email_size,log_file,log_key_id)
                FROM '/encrypted_vol/jupyter_data/airflow/data/2019-08-22.csv'
                DELIMITER ',' CSV HEADER;''')


In [51]:
cur.execute('SELECT * FROM email_logs LIMIT 2')

In [52]:
cur.fetchall()

[('aRFWSXGwyJiTxPGMwzZSJcdRjWtpjCqB',
  datetime.datetime(2019, 8, 23, 0, 0, 1),
  'nsanchez@gilbert-wade.com',
  "['rodriguezsabrina@yahoo.com', 'ymcbride@hotmail.com', 'gberry@hotmail.com', 'karen09@hotmail.com']",
  'Position or expect someone radio have.',
  "['H:/him/technology/memory/many/sport/claim.inf']",
  'AND',
  'WHOLE_MAINTAIN_STOP',
  1,
  4,
  26100063,
  '201901.gz',
  None),
 ('RdvnVgbwKGlywgFHTQdomRmFHVXPkRwt',
  datetime.datetime(2019, 8, 23, 0, 0, 4),
  'phillipsstacey@gmail.com',
  "['cjenkins@hoffman.org', 'allen94@gmail.com', 'qunderwood@gmail.com', 'randy33@rodriguez-bennett.com', 'lindsaymendoza@rodriguez.com', 'rebeccaflowers@gmail.com', 'hicksjesus@cooper-jenkins.com', 'donna11@barron-pace.biz', 'nancyzuniga@hotmail.com', 'edwin93@doyle.com', 'kathy98@flores.com', 'dunnhunter@gmail.com', 'gkelly@gmail.com', 'vflores@mendez.net', 'xhanson@greer.com']",
  'Subject few wide performance size.',
  "['Z:/thought/while/management/few.jar']",
  'HEAR',
  'TAKE_DECAD

In [ ]:
get_table_col_names(conn, 'employees')

### Insert into blended table

In [59]:
cur.execute('rollback')

In [48]:
cur.execute('DROP TABLE IF EXISTS blended_email_logs')

In [49]:
cur.execute('''CREATE TABLE IF NOT EXISTS blended_email_logs (
                guid varchar, 
                "datetime" TIMESTAMP, 
                sender varchar, 
                recipients varchar, 
                subject varchar, 
                attachments varchar, 
                rule varchar, 
                action varchar, 
                attach_cnt int, 
                rcpt_cnt int, 
                email_size int, 
                log_file varchar, 
                log_key_id varchar, 
                name varchar,
                phone_number varchar,
                address varchar,
                PRIMARY KEY(guid, sender));''')

In [104]:
cur.execute('''insert into blended_email_logs(
                        guid,
                        datetime,
                        sender,
                        recipients,
                        subject,
                        attachments,
                        rule,
                        action,
                        attach_cnt,
                        rcpt_cnt,
                        email_size,
                        log_file,
                        log_key_id,
                        name,
                        phone_number,
                        address
)
SELECT 
  el.guid,el.datetime,el.sender,el.recipients,el.subject,el.attachments,el.rule,el.action,el.attach_cnt,el.rcpt_cnt,el.email_size,el.log_file,el.log_key_id,
  e.name,e.phone_number,e.address
FROM email_logs el
LEFT JOIN employees e ON el.sender = e.email''')


In [109]:
cur.execute('''SELECT count(*) FROM blended_email_logs WHERE datetime::date = date '2019-08-21' ''')

In [110]:
cur.fetchone()

(7411,)

In [111]:
cur.execute('''SELECT count(*) FROM blended_email_logs''')

In [112]:
cur.fetchone()

(22260,)

In [193]:
conn.close()